In [1]:

from google.colab import files
uploaded = files.upload()



ModuleNotFoundError: No module named 'google'

# DATA FEATURES

•Pregnancies: Number of pregnancies of the patient.

•Glucose: Glucose of the patient.

•BloodPressure: Blood pressure of the patient.

•SkinThickness: Skin thickness of the patient.

•Insulin: Insuling of the patient.

•BMI: BMI of the patient.

•DiabetesPedigreeFunction: Diabetes pedigree function of the patient.

•Age: Age of the patient.

•Outcome: (TARGET VARIABLE) Whether the patient is likely to suffer from diabetes or not.

# EXPLORATORY DATA ANALYSIS

## IMPORTING NECESSARY LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d # used in 3D plotting

import matplotlib.pyplot as plt # data visualization
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns #

In [ ]:
df = pd.read_csv('diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## CORRELATION MATRIX

In [ ]:
corr = df.corr()
max(corr)
plt.figure(figsize=(14,12))
sns.heatmap(corr, cmap='YlGnBu', annot=True) 
plt.show()

## DATA VISUALIZATIONS

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(x = df['BloodPressure'], y = df['Insulin'])
plt.show()

In [ ]:
sns.boxplot(x='BMI', data=df)

In [ ]:
sns.countplot(x='Pregnancies', data=df)

In [ ]:
plt.figure(figsize=(10,6))
sns.violinplot(x='Outcome', y='Glucose', data=df)

In [ ]:
sns.scatterplot(x='BMI', y='Age', hue='Outcome', data=df, palette='Set1')
plt.xlabel('BMI')
plt.ylabel('Age')

## TRAIN TEST SPLITTING

In [ ]:
X= df.drop(['Outcome'], axis=1)
y = df['Outcome']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 82)

In [ ]:
pip install lazypredict

# CHECKING ACCURACY OF THE MODEL

In [ ]:
import lazypredict

### importing LazyClassifier for classification problem
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)


In [ ]:
models,predictions = clf.fit(X_train, X_test, y_train, y_test)


print(models)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
lda = LinearDiscriminantAnalysis()

# Define the hyperparameters to tune
param_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': ['auto', None],
    'n_components': [None, 1, 2, 3]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(lda, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

# Instantiate a new LDA model with the optimized hyperparameters
lda = LinearDiscriminantAnalysis(**grid_search.best_params_)

# Fit the model on the training data
lda.fit(X_train, y_train)

# Evaluate the model on the testing data
score = lda.score(X_test, y_test)
print(f"Testing accuracy: {score}")

In [ ]:
y_pred = lda.predict(X_test)


# PERFORMANCE MEASURE

## CLASSIFICATION REPORT

In [ ]:
from sklearn.metrics import classification_report
print("Accuracy on unknown data is",classification_report(y_test,y_pred))



In [ ]:
def get_classification_report(y_test, y_pred):
    
    from sklearn import metrics
    report = metrics.classification_report(y_test, y_pred, output_dict=True)
    df_classification_report = pd.DataFrame(report).transpose()
    df_classification_report = df_classification_report.sort_values(by=['f1-score'], ascending=False)
    return df_classification_report

s= get_classification_report(y_test, y_pred)
s

## CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Compute the classification report
cr = classification_report(y_test, y_pred)

# Plot the confusion matrix heatmap
sns.heatmap(cm, annot=True, cmap='PuOr', fmt='g', xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()



# PICKLING THE MODEL

In [ ]:
import pickle
with open('diabetes.pkl', 'wb') as files:
    pickle.dump(lda,files)